# Статистический анализ данных

*Приложение предназначено для проведения статистического анализа данных. Для корректной работы программы необходимо подгрузить Excel-файл с данными, которые могут содержать как численные, так и категориальные значения. Количество столбцов с параметрами неограничено.*

In [3]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import VBox
from ipywidgets.widgets.interaction import show_inline_matplotlib_plots
import scipy.stats as st
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
class App:
    
    def __init__(self):
        self.btn_upload = widgets.FileUpload(description = 'Загрузить')
        self.btn_run = widgets.Button(description = 'Сводные таблицы')
        self.btn_plot = widgets.Button(description = 'Построить графики')
        self.btn_clean = widgets.Button(description = 'Очистить')
        self.out_pl = widgets.Output()
   

    def on_click_tables(self, change):
        try:
            df = pd.read_excel(self.btn_upload.data[-1], engine = 'openpyxl')
            
            df.dropna(inplace=True)

            cat = [i for i in df.columns if df[i].dtype == 'O']

            self.out_pl.clear_output()

            with self.out_pl:

                print("\033[1m" + 'Значения P10:' + "\033[0m")
                display(df.groupby(cat).agg(lambda x: st.t.ppf(0.10, df=len(x)-1, loc=np.mean(x), scale=st.sem(x))))
                print(' ')

                print("\033[1m" + 'Значения P50:' + "\033[0m")
                display(df.groupby(cat).agg(lambda x: st.t.ppf(0.50, df=len(x)-1, loc=np.mean(x), scale=st.sem(x))))
                print(' ')

                print("\033[1m" + 'Значения P90:' + "\033[0m")
                display(df.groupby(cat).agg(lambda x: st.t.ppf(0.90, df=len(x)-1, loc=np.mean(x), scale=st.sem(x))))
                print(' ')
        except:
            self.out_pl.clear_output()
            with self.out_pl: print('Вероятно, отсутствуют данные либо подгружен некорректный файл.')
        
        
    def on_click_graphs(self, change):
        try:
            df = pd.read_excel(self.btn_upload.data[-1], engine = 'openpyxl')
            
            df.dropna(inplace=True)
            
            num = [i for i in df.columns if df[i].dtype != 'O']

            self.out_pl.clear_output()
            with self.out_pl:
                for i, j in enumerate(num):
                    fig = plt.figure(figsize=(7,4), clear = True)
                    x = df[j]
                    sns.distplot(x)
                    plt.axvline(st.t.ppf(0.10, df=len(x)-1, loc=np.mean(x), scale=st.sem(x)), c='red', label='P10')
                    plt.axvline(st.t.ppf(0.50, df=len(x)-1, loc=np.mean(x), scale=st.sem(x)), c='blue', label='P50')
                    plt.axvline(st.t.ppf(0.90, df=len(x)-1, loc=np.mean(x), scale=st.sem(x)), c='green', label='P90')
                    plt.legend(loc = 'upper right')
                show_inline_matplotlib_plots()
                plt.show()
        except:
            self.out_pl.clear_output()
            with self.out_pl: print('Вероятно, отсутствуют данные либо подгружен некорректный файл.')
                    
        
    def on_click_clean(self, change):
        self.out_pl.clear_output()

In [5]:
app = App()

In [6]:
app.btn_run.on_click(app.on_click_tables)
app.btn_plot.on_click(app.on_click_graphs)
app.btn_clean.on_click(app.on_click_clean)

In [7]:
display(VBox([app.btn_upload, app.btn_run, app.btn_plot, app.btn_clean, app.out_pl]))